In [ ]:
from __future__ import absolute_import
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy
from pycuda.curandom import rand as curand

a_gpu = curand((50,))
b_gpu = curand((50,))

from pycuda.elementwise import ElementwiseKernel
lin_comb = ElementwiseKernel(
        "float a, float *x, float b, float *y, float *z",
        "z[i] = my_f(a*x[i], b*y[i])",
        "linear_combination",
        preamble="""
        __device__ float my_f(float x, float y)
        { 
          return sin(x*y);
        }
        """)

c_gpu = gpuarray.empty_like(a_gpu)
lin_comb(5, a_gpu, 6, b_gpu, c_gpu)

import numpy.linalg as la
assert la.norm(c_gpu.get() - numpy.sin((5*a_gpu*6*b_gpu).get())) < 1e-5

In [ ]:
%timeit -n 1000 cuda.memset_d32(sdata_gpu, 0, sdata.size); image_mask_split(mask_idx_gpu, image_gpu, np.int32(64), np.int32(64), sdata_gpu,  block=(1, 32, 32), grid=(bs, 1, 1), shared=0)

In [ ]:
## Sample source code from the Tutorial Introduction in the documentation.
%matplotlib inline
from __future__ import print_function
from __future__ import absolute_import
import pycuda.driver as cuda
import pycuda.autoinit  # noqa
from pycuda.compiler import SourceModule

from beesgrid.generate_grids import BlackWhiteArtist, MASK, MASK_BLACK, \
    MASK_WHITE, GridGenerator, MaskGridArtist
    
import beesgrid.generate_grids as gen_grids

import numpy
import numpy as np
import matplotlib.pyplot as plt
import time



kernel_code = """
// Kernel definition

#define MASK_LEN 28
#define WARP_SIZE 32


// TODO: include beesgrid headers for mask enum
enum MASK {
    INNER_BLACK_SEMICIRCLE,
    CELL_0_BLACK = 1,
    CELL_1_BLACK,
    CELL_2_BLACK,
    CELL_3_BLACK,
    CELL_4_BLACK,
    CELL_5_BLACK,
    CELL_6_BLACK,
    CELL_7_BLACK,
    CELL_8_BLACK,
    CELL_9_BLACK,
    CELL_10_BLACK,
    CELL_11_BLACK,
    IGNORE = 128,
    CELL_0_WHITE = IGNORE + 1,
    CELL_1_WHITE = IGNORE + 2,
    CELL_2_WHITE = IGNORE + 3,
    CELL_3_WHITE = IGNORE + 4,
    CELL_4_WHITE = IGNORE + 5,
    CELL_5_WHITE = IGNORE + 6,
    CELL_6_WHITE = IGNORE + 7,
    CELL_7_WHITE = IGNORE + 8,
    CELL_8_WHITE = IGNORE + 9,
    CELL_9_WHITE = IGNORE + 10,
    CELL_10_WHITE = IGNORE + 11,
    CELL_11_WHITE = IGNORE + 12,
    OUTER_WHITE_RING = IGNORE + 20,
    INNER_WHITE_SEMICIRCLE = IGNORE + 21
};
__device__ const int MASKS_INDICIES[]= {
    INNER_BLACK_SEMICIRCLE, CELL_0_BLACK, CELL_1_BLACK, CELL_2_BLACK, CELL_3_BLACK,
    CELL_4_BLACK, CELL_5_BLACK, CELL_6_BLACK, CELL_7_BLACK, CELL_8_BLACK, CELL_9_BLACK,
    CELL_10_BLACK, CELL_11_BLACK, IGNORE, CELL_0_WHITE, CELL_1_WHITE, CELL_2_WHITE,
    CELL_3_WHITE, CELL_4_WHITE, CELL_5_WHITE, CELL_6_WHITE, CELL_7_WHITE, CELL_8_WHITE,
    CELL_9_WHITE, CELL_10_WHITE, CELL_11_WHITE, OUTER_WHITE_RING, INNER_WHITE_SEMICIRCLE
};



__device__ inline int index2(const int dim1, const int y, const int x) {
    return dim1*y + x;
}

__device__ inline int index3(const int dim2, const int dim1,
               const int z, const int y, const int x) {
    return dim2*dim1*z + dim1*y + x;
}
__device__ inline int index4(const int dim3, const int dim2, const int dim1,
               const int w, const int z, const int y, const int x) {
    return dim3*dim2*dim1*w + dim2*dim1*z + dim1*y + x;
}

extern "C" {

__global__ void to_sum_var_count(const float * reduced, const int n,
                                          float * sum, float * var, float * count) {
    const int block = blockIdx.x * blockDim.x;
    const int tid = threadIdx.x;
    const int idx = block + tid;
    int new_pos = MASK_LEN*floorf(idx / float(3*MASK_LEN)) + idx % MASK_LEN;
    if(idx % 3*MASK_LEN < MASK_LEN) {
        sum[new_pos] = reduced[idx];
    } else if(idx % 3*MASK_LEN < 2*MASK_LEN) {
        new_pos += MASK_LEN;
        var[new_pos] = reduced[idx] - pow(reduced[idx - MASK_LEN], 2);
    } else {
        new_pos += 2*MASK_LEN;
        count[new_pos] = reduced[idx];
    }
}

__global__ void image_mask_split(const float * mask, const float * image,
                                 const int bs, const int N, float * o_split)
{
    const int block_r = blockIdx.y * blockDim.y;
    const int block_c = blockIdx.z * blockDim.z;
    const int r = block_r + threadIdx.y;
    const int c = block_c + threadIdx.z;
    const int b = blockIdx.x;
    if (b < bs && r < N && c < N) {
        const int s_idx_base = index4(bs, N, N, 0, b, r, c);
        const int next_mask_offset = bs*N*N;
        const int offset = bs*N*N*MASK_LEN;
        const int index = index3(N, N,
                                 b, r, c);

        for(int i = 0; i < MASK_LEN; i++) {
            const int s_idx = s_idx_base + i*next_mask_offset;
            if(mask[index] == MASKS_INDICIES[i]) {
                o_split[s_idx] = image[index];
                o_split[s_idx + offset] = pow(image[index], 2);
                o_split[s_idx + 2*offset] = 1;
            }
        }
    }
}

__global__ void image_mask_split_grad(const float * mask, const float * image, 
                                 const float * out_grad_sum, const float * out_grad_pow,
                                 const int bs, const int N, float * grad)
{
    const int block_r = blockIdx.y * blockDim.y;
    const int block_c = blockIdx.z * blockDim.z;
    const int sr = block_r + threadIdx.y;
    const int sc = block_c + threadIdx.x;

    const int b = blockIdx.x;
    const int r = 2*sr;
    const int c = 2*sc;
    const int N2 = N/2;
    const int s_idx_base = index4(bs, N2, N2, 0, b, sr, sc);
    const int next_mask_offset = bs*N2*N2;
    const int offset = bs*N2*N2*MASK_LEN;
    const int index = index3(N, N, b, r, c);
    if (b < bs && r + 1 < N && c + 1 < N && sr < N2 && sc < N2) {
        if (index < bs*N*N) {
            float mySum = 0;
            for(int i = 0; i < MASK_LEN; i++) {
                const int s_idx = s_idx_base + i*next_mask_offset;
                if(mask[index] == MASKS_INDICIES[i]) {
                     mySum += 2*image[index]*out_grad_pow[s_idx + offset] + out_grad_sum[s_idx];
                }
            }
            grad[index] = mySum;
        }
    }
}
}"""

mod = SourceModule(kernel_code, no_extern_c=1)

#    //if(blockIdx.z == 0 && blockIdx.y == 0 && blockIdx.x == 0 &&
#    //   threadIdx.z == 0 && threadIdx.y == 0 && threadIdx.x == 0) {
#    //    black_mean = black_sum;
#    //}
image_mask_split = mod.get_function("image_mask_split")
        

def masks(batch_size):
    batch_size += 64 - (batch_size % 64)
    generator = GridGenerator()
    artist = MaskGridArtist()
    for masks in gen_grids.batches(batch_size, generator, artist=artist,
                                   scales=[1.]):
        yield masks[0].astype(np.float32)

bs = 64 # 1024
image = np.random.sample((bs, 1, 64, 64)).astype(np.float32)
mask_idx = next(masks(1))
sdata = np.zeros((3*len(MASK), bs, 1, 64, 64), dtype=np.float32)

black_mean = np.zeros((bs, 1, 32, 32)).astype(np.float32)


print((image.shape, mask_idx.shape, black_mean.shape))
print((image.dtype, mask_idx.dtype, black_mean.dtype))

def to_gpu(arr):
    gpu_arr = cuda.mem_alloc(arr.nbytes)
    cuda.memcpy_htod(gpu_arr, arr)
    return gpu_arr
        
sdata_gpu = to_gpu(sdata)
mask_idx_gpu = to_gpu(mask_idx)
image_gpu = to_gpu(image)
image_mask_split(cuda.In(mask_idx), cuda.In(image), np.int32(bs), np.int32(64), cuda.InOut(sdata), 
     block=(1, 32, 32), grid=(bs, 2, 2))

      
rdata = np.ones((bs, 3*len(MASK)), dtype=np.float32)
rdata_gpu = to_gpu(rdata)

start = time.time()
#reduce_sum(cuda.In(sdata), np.uint32(32*32), cuda.InOut(rdata),
#     block=(16*8, 1, 1), grid=(rdata.size, 1, 1), shared=16*16*16)
print("run cuda kernel in {}s".format(time.time() - start))

#cuda.memcpy_dtoh(sdata, sdata_gpu)
#black_mean = sdata[:, :, :, :, :13].sum(axis=4)
#np_black_mean = np.zeros_like(image)        
#black_idx = mask_idx < MASK["IGNORE"]
#np_black_mean[black_idx] = image[black_idx]
#plt.subplot(131)
#plt.imshow(reduce_data[0, 0, :, :, 0])
#plt.colorbar()        

ignore = np.zeros((64, 64))
ignore_idx = mask_idx[63, 0] == MASK["IGNORE"]
ignore[ignore_idx] = image[63, 0][ignore_idx]
plt.subplot(131)
plt.imshow(ignore)
plt.subplot(132)
plt.imshow(sdata[13, 63, 0, :, :])

np_black_mean = np.zeros_like(image)        
white_idx = mask_idx > MASK["IGNORE"]
np_black_mean[white_idx] = 1
plt.subplot(133)
plt.imshow(np_black_mean[63, 0, :, :])
        
#plt.subplot(133)
#plt.imshow(np_black_mean[0][0])
plt.show()
np.set_printoptions(threshold=np.nan)
# print(sdata[13, 63, 0, :, :])
print(ignore.sum())
print(sdata.sum(axis=(2, 3, 4))[13, 63])
total_sum = 0
for k in sorted(MASK):
    s = (mask_idx == MASK[k]).sum()
#    print((k, MASK[k], s))
    total_sum += s
m = mask_idx.copy()
for k, v in MASK.items():
    m[mask_idx == v] = -1

#print(total_sum)
#print(64*64*64)


In [ ]:
print(type(sdata.size))

In [ ]:
import numpy, theano
import theano.misc.pycuda_init
from pycuda.compiler import SourceModule
import theano.sandbox.cuda as cuda
from theano.sandbox.cuda.type import CudaNdarrayType
from theano.misc.pycuda_utils import to_gpuarray, to_cudandarray
class SplitMaskGrad(theano.Op):

    __props__ = ()

    def make_node(self, mask_idx, image, og_sum, og_pow, og_count):
        contiguouse = lambda x:  cuda.basic_ops.gpu_contiguous(
           cuda.basic_ops.as_cuda_ndarray_variable(x))
        
        mask_idx = contiguouse(mask_idx)
        image = contiguouse(image)
        og_sum = contiguouse(og_sum)
        og_pow = contiguouse(og_pow)
        og_count = contiguouse(og_count)


        output_type = CudaNdarrayType(broadcastable=(False,)*4)
        return theano.Apply(self, [mask_idx, image, og_sum, og_pow, og_count], 
                            [output_type()])
    

    def make_thunk(self, node, storage_map, _, _2):
        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]
        mod = SourceModule(kernel_code, no_extern_c=1)
        image_mask_split_grad = mod.get_function("image_mask_split_grad")

        def thunk():
            grad = outputs[0][0]
            mask_idx = inputs[0][0]
            image = inputs[1][0]
            batch_size = mask_idx.shape[0]
            shape = (batch_size, 1, 64, 64)
            if grad is None or grad.shape != shape:
                grad = cuda.CudaNdarray.zeros(shape)
            sdata = to_gpuarray(sdata)
            grid = (batch_size, 1, 1)
            mask_idx = to_gpuarray(mask_idx)
            image = to_gpuarray(image)
            image_mask_split_grad(mask_idx, image, np.int32(batch_size),
                             np.int32(64), sdata, block=(32, 32, 1), grid=grid, shared=0)
            return [theano.gradient.grad_undefined(), to_cudandarray(grad)]
        
        return thunk


class SplitMask(theano.Op):

    __props__ = ()

    def make_node(self, mask_idx, image):
        contiguouse = lambda x:  cuda.basic_ops.gpu_contiguous(
           cuda.basic_ops.as_cuda_ndarray_variable(x))
        mask_idx = contiguouse(mask_idx)
        image = contiguouse(image)
        assert mask_idx.dtype == "float32"
        assert image.dtype == "float32"
        output_type = CudaNdarrayType(broadcastable=(False,)*5)
        print(image.type())
        return theano.Apply(self, [mask_idx, image], 
                            [output_type(), output_type(), output_type()])
    

    def make_thunk(self, node, storage_map, _, _2):
        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]
        mod = SourceModule(kernel_code, no_extern_c=1)
        image_mask_split = mod.get_function("image_mask_split")

        def thunk():
            sdata = outputs[0][0]
            mask_idx = inputs[0][0]
            image = inputs[1][0]
            batch_size = mask_idx.shape[0]
            shape = (3*len(MASK), batch_size, 1, 32, 32)
            if sdata is None or sdata.shape != shape:
                sdata = cuda.CudaNdarray.zeros(shape)
            sdata = to_gpuarray(sdata)
            grid = (batch_size, 1, 1)
            mask_idx = to_gpuarray(mask_idx)
            image = to_gpuarray(image)
            image_mask_split(mask_idx, image, np.int32(batch_size),
                             np.int32(64), sdata, block=(32, 32, 1), grid=grid, shared=0)
            sdata = to_cudandarray(sdata)
            m = len(MASK)
            outputs[0][0] = sdata[:m]
            outputs[1][0] = sdata[m:2*m]
            outputs[2][0] = sdata[2*m:]

        return thunk
    
    def grad(self, inputs, output_gradients):
        grad_ins = inputs + output_gradients
        return SplitMaskGrad()(*grad_ins)
        
        
split_mask = SplitMask()

In [ ]:
import theano.tensor as T
t_mask = theano.tensor.tensor4()
t_img = theano.tensor.tensor4()
raw_sum, raw_pow, raw_count = split_mask(t_mask, t_img)
mask_sum = T.sum(raw_sum, axis=[2, 3, 4])
mask_pow = T.sum(raw_pow, axis=[2, 3, 4])
mask_count = T.sum(raw_count, axis=[2, 3, 4])
mean = T.switch(T.eq(mask_count, 0), T.zeros_like(mask_sum), mask_sum / mask_count)
var = T.switch(T.eq(mask_count, 0), T.zeros_like(mask_sum), mask_pow / mask_count - mean**2 )


split_fn = theano.function([t_mask, t_img], [raw_sum, raw_pow, raw_count])
sum_fn = theano.function([t_mask, t_img], [mask_sum, mask_pow, mask_count])
mean_var_fn = theano.function([t_mask, t_img], [var, mean])

In [ ]:
%timeit -n 200 sum_fn(mask_idx, image)

In [ ]:
%timeit -n 200 split_fn(mask_idx, image)

In [ ]:
%timeit -n 200 mean_var_fn(mask_idx, image)

In [ ]:
x = mean_var_fn(mask_idx, image)
hx = np.asarray(x)
print(hx.shape)


In [ ]:
idx = mask_idx[63] == MASK["IGNORE"]
print(image[63][idx].var())
print(image[63][idx].mean())

In [ ]:
x = sum_fn(mask_idx, image)


In [ ]:
T.grad(T.mean(mean + var), t_img)